In [1]:
from keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle

In [2]:
model = load_model('model/LSTM model.h5')

In [3]:
# loading
with open('model/t.pickle', 'rb') as handle:
    t = pickle.load(handle)

In [8]:
# 불러올때    
with open('data/result.txt', 'rb') as f:
    result = pickle.load(f)

result_gensim_input = [_.split() for _ in result if _ != '']
result_tokenizer_input = [v for i, v in enumerate(result) if i%2 == 0 and v != '']
ebs_in_result_for_getting_max_len = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != []]
ebs_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 0 and v != [] ]
google_in_result_gensim_input = [v for i, v in enumerate(result_gensim_input) if i%2 == 1 and v != []]

In [29]:
print(len(result_gensim_input))
print(len(result_tokenizer_input))
print(len(ebs_in_result_for_getting_max_len))
print(len(ebs_in_result_gensim_input))
print(len(google_in_result_gensim_input))

4162
2081
2081
2081
2079


In [4]:
def sentence_generation(model, t, current_word, n): # 모델, 토크나이저, 현재 단어, 반복할 횟수
    init_word = current_word # 처음 들어온 단어도 마지막에 같이 출력하기위해 저장
    sentence = ''
    for _ in range(n): # n번 반복
        encoded = t.texts_to_sequences([current_word])[0] # 현재 단어에 대한 정수 인코딩
        encoded = pad_sequences([encoded], maxlen=23, padding='pre') # 데이터에 대한 패딩
        result = model.predict_classes(encoded, verbose=0)
    # 입력한 X(현재 단어)에 대해서 y를 예측하고 y(예측한 단어)를 result에 저장.
        for word, index in t.word_index.items(): 
            if index == result: # 만약 예측한 단어와 인덱스와 동일한 단어가 있다면
                break # 해당 단어가 예측 단어이므로 break
        current_word = current_word + ' '  + word # 현재 단어 + ' ' + 예측 단어를 현재 단어로 변경
        sentence = sentence + ' ' + word # 예측 단어를 문장에 저장
    # for문이므로 이 행동을 다시 반복
    sentence = init_word + sentence
    return sentence

In [64]:
ebs_final_comparison = []
google_final_comparison = []
for _ in range((int(len(result)/2)-1)+1):
    e = result[_*2]
    g = result[_*2+1]
    if e != '' and g != '':
        ebs_final_comparison.append(e)
        google_final_comparison.append(g)

In [65]:
len(ebs_final_comparison)

2081

In [66]:
len(google_final_comparison)

2081

In [67]:
for e, g in zip(ebs_final_comparison, google_final_comparison):
    n = len(g.split())
    first_w = g.split()[0]
    print('ebs')
    print(e)
    print()
    print('google')
    print(g)
    print()
    print('predict')
    print(sentence_generation(model, t, first_w, n-1))
    print()
    print('*****'*10)

ebs
 반갑습니다

google
 반갑습니다

predict
반갑습니다

**************************************************
ebs
 이비에스 친구 여러분 여러분 함께 중학 뉴런 수학 삼하를 강의 된손석민 입니다

google
 이비에스  여러분 여러분 함께 중학 뉴런 수학 삼할을 강의 하기됨손성민 입니다

predict
이비에스 여러분 답 그냥 우리 여러분 이렇게 구해주시되겠습니다 했어요 여러분 여러분

**************************************************
ebs
 학년 뉴런 통해 서도 만났었는데요 여러분 이렇게 다시 만나게 되어서 정말 반갑습니다

google
 학년 이런을 통해 서도 만났었는데 여러분 이렇게 다시 만나게 되어서 정말 반갑습니다

predict
학년 개념 보조선그 개념 걸 좀 여러분 많이 끝나면안되 해 우리 여기

**************************************************
ebs
 앞 삼하 과정 어떻게 진행 되는지에 대해 서는 여러분 오리엔테이션 강의 통해 확인 할수있을거고요 오리엔테이션 강의 반드시 듣고오셔야 여러분 이삼 학년 과정 이하과정특히 어떻게 수업 되는지 확인 할수있을 겁니다

google
 앞 삼 과정 어떻게 진행 되는지에 대해 서는 여러분 오리엔테이션 강의 통해 확인 할수있을거구요 오리엔테이션 강의 반드시 듣고오셔야 여러분 이삼 학년 과정 하마종이접기 어떻게 수업 되는지 확인 할수있을 겁니다

predict
앞 뭔가 우리 우리 형성평가 선생님 함께 마무리 할게요 마무리 일단 여러분 앞 풀이 되겠습니다 개념 해서 삼 길이 이렇게 해서 풀 사인 사인 여러분 이제 선생님 삼각형 넓이

**************************************************
ebs
 일 단삼 학년 과정 같은 경우 크게 삼 개 대단원 이루어져있는데요 첫 삼각비 단원두번째 가원 성질 세번 통계 이렇게 이루어져있습니다

goog

오늘 뭔가 우리 문제 마무리 상관없어요 사실 여러분 교재 여러분 또 함께 하고있습니다 예제 사 에이치 오늘 내용 굉장히 선생님 그림 보자마자 우리 형성평가 그림 삼십 같이 이렇게 뭘 봤으면 할수있겠는데 각 넓이 한다고 코사인 구 이비 에이길이 삼각형 이렇게 들어가

**************************************************
ebs
 선생님 정확한 개념 시작 해볼테니까 집중 해서 들어주시기 바랍니다

google
 선생님 정확한 개념 시작 해볼테니까 집중 해서 들어주시기 바랍니다

predict
선생님 제 선생님 바람 하자 삼각비표 첫 오늘 사인

**************************************************
ebs
 일단 기본 삼각비 앞 쭉 이야기 할텐데 가장 전제 조건 직각삼각형 이야기 합니다

google
 일단 기본 삼각비 앞 쭉 이야기 할텐데 가장 전제 조건 직각삼각형 이야기 합니다

predict
일단 문제 코사인 엑스 에이 미터 결국 뭐 마주 뭐 밑변 에이 그러면

**************************************************
ebs
 직각삼각형 삼각비 이야기 하는구나 앞 삼각비 피타고라스 정리 때 직각삼각형 우리 따졌듯주로 직각삼각형 삼각비 이야기 할거예요 그런 데이 직각삼각형 그려놓 고우리 생각 해보면 직각삼각형 이렇게 그려놓고기호 각 각의 꼭짓점 에이 비씨 잡아 보겠습니다

google
 직각삼각형 삼각비 이야기 하는구나 앞 삼각비 피타고라스 정리 때 직각삼각형 우리 다른뜻 직각삼각형 상각비 이야기 할거야 그런 데이 직각삼각형 그려놓 고우리 생각 해보면 직각삼각형 이렇게 그려놓고자기를 각 각의 꼭지점 에이 비씨 이렇게번잡 보겠습니다

predict
직각삼각형 처음 엑스 같은 문제 여러분 함께 수업 에이오를구 고난 작은 뉴런 여러분 쭉걷고있는데요여러분 삼 시간 만나서 시간 보세요 나와요 시간 오늘 페이지 숙제 얼마 레벨 얘기 여러분 되겠습니다 사인 왜냐하면 

탄젠트 에이 에이치 해주면 오도 바로 양변 일단 이렇게 사 높이 사인 돼서 여기 삼각비 탄젠트 뭐 스쳐 바로 볼게요 탄젠트 코사인 탄젠트 또는 이분 높이 사인 여러분 여기 삼분 비 루트 새로운 탄젠트 할수있구 돼서 얘 많이 우리 뭐 그림 중요한건 밑변 뭐 할수 루트

**************************************************
ebs
 그러니까 결국세 변 에두 변 길이 의비 뽑는 방법 육 가지 사인 코사인 탄젠트 삼가지를 우리 공부 거구사인 빗변 높이 코사인빗변 밑변 탄젠트 밑변

google
 그러니까 결국세번에두 변 에두 변 길이 의비 뽑는 방법 육 가지 사인 코사인 탄젠트 세가지를 우리 공부 하는거구나쌓이는주변브래드핏쌓이는 빗변 높이  밑변 탄젠트 밑변

predict
그러니까 사인 코사인 에이 에이 뭐 할수있어 용이 마주 보이는 그것 여기 결국 삼각형 우리 높이 뭐 보는거예요 된다는얘기 까 뭘 사인이겠구나뭐 코사인

**************************************************
ebs
 여기 여러분 이기 억해 두시 되는거예요이제 그럼이 기본 내용 가지 고우리 문제 어떻게 적용 되는지 봐야되 겠죠함께예제살펴보겠습니다

google
 여기 얼음 이기 억해 두시 되는거야 그럼이 기본 내용 가지 고우리 문제 어떻게 적용 되는지 봐야되 겠다

predict
여기 꼭 탄젠트 삼십 코사인 에이 에이 에이치 뭐 보는 경우 빗변밑변의비주어져있으 양변 탄젠트 코사인 탄젠트 거

**************************************************
ebs
 예제 일 번함께풀이 시작 해보도록 할텐데요 삼각비값구 하기 확인 문제 별로 차이 없어 보이긴 합니다

google
  일 일벌써망뿌리 시작 해보도록 할텐데요 숙박비값구 하기 확인 문자 별로 차이 없어 보 합니다

predict
일 문제 뭐 코사인 엑스 이각 되는 우리 보 삼각형 원래 뭐 이번 삼각비

**********************

이건우 데이 때 오늘 개념 제한 여러분 여러분 시작 해볼텐데 지점 삼각비

**************************************************
ebs
 맞죠그럼이각이구십도도 입니다 그러면 내각의합 백팔십 가되 기 위해 서는 결국 이각비건삼각형내각의합백팔십나머지둘의합 빼주 되죠 구십 마이너스 엑스 구십 의합빼주거구십더하기구십는백팔십빼서없어지고마이너스마이너스 엑스 엑스 돼서이각엑스돼그러면결국이각 엑스 일때이각 엑스 것알수있죠왜걸구할거냐결론본다면 탄젠트 엑스 구하는게문제근데이엑스모르잖아요그러면탄젠트엑스삼각형지금엑스있으니탄젠트 뭐 뭐였어밑변높이였었죠기준각마주보는이변높이고이밑변거든요뭐둘다몰라 그러니까 값구할수있어없어없어요우리어차피직각삼각형닮음일때길이의비가모같다고했죠그러니까결국탄젠트엑스걸엑스놓고밑변높이하나이각엑스임알나면이제잘 보세요 이안있는선쳐다보지말고전체큰삼각형보겠습니다여기보세요엑스대해마주보는 변 뭐기준각대해마주보는변높이겠고이게빗변잖아요그렇죠직각마주보는빗변이변금밑변입니다

google
  입니다 그러면 가계약 백팔십 가되 기 위해 서는 결국 이갑건삼각형외각백팔십나머지렙빼주되죠구십마이너스엑스구십 빼주  구십 마이너스 엑스 구십 더하기구십백팔십빼서없어지고마이너스마이너스했으니까엑스되서가게엑스그러면결국 엑스 엑스 구하는게문제근데이 엑스 모르잖아요그러면탄젠트 엑스  탄젠트 엑스 있으니보였어및높이왔었죠기준가게마주보이는이변높이편이거든요 뭐 둘다몰라그러니까값구할수있어없어없어요우리어차피직각삼각형닮음일대길이의비못갔다그랬어 그러니까 결국탄젠트엑스걸엑스놓고밑변보내높이일이더하기엑스이름알나면이제잘보세요요안있는쳐다보지말고전체큰상가통보겠습니다여기 보세요 엑셀마주보이는변뭐기준가게대해마주보이는벽높이됐고이있잖아요죠집가게마시고이름 변 이변금편입다

predict
입니다 여러분 우리 오늘 수업 마무리 할 일 여러분 삼 번은 여러분 여러분 사실 문제 이렇게 여러분 가장 있죠 삼하를 내용 여러분 이점 구 되어야 할게요 여러분 것 용해서우리 느껴질거라 우리 여러분 각

**********

왜 선생님 오늘 선생님 오늘 풀이 점사 답변 공부 각 답얻을수있다는얘기사인삼십도 하죠느낌오나요먼저그걸여러분좀이해해돼여기하나세워놓고탑있고 길이 용해서우리 그대로 이구해 이구해 용해 이각 코사인 갈게요 코사인 이비 내각 직각삼각형

**************************************************
ebs
 그럼이 제구 자하게사인에이 더하기 코사인에이 기준 각거 다시 한번 써 볼까 마주 보는 변 높이 였고 이변 밑변이변금 빗변 잖아사인 뭔 데 빗변 높이 오분 사바갈수있겠죠코사인뭔데빗변밑변오분의삼그러니까결과뭐된다

google
  제구 자는게쌓이네 더하기 구사이사이 기준 가게것 다시 한번 써 볼까 마주 보이는 변 높이  이변 몇번의견있으면불편하잖아사인뭔데 빗변 높이오븐바로갈수있겠죠콜사인 뭔 데 빗변 밑변 오분 의삼그러니까결과먹어야된다

predict
제구 우리 문제 마무리 풀어도 여기 여러분 사실 앞 수학 이제 난이도 반드시 되겠습니다 시간 여러분 때 건물 선생님 돼있어 마무리 정답 삼 이런

**************************************************
ebs
 오분 칠 선택 지에서 정답 몇번 삼번 이렇게 여러분들이구할수있으면 되겠죠 다음유제육번짝이뤄서가니까문제삼각비주어져있으면우리 그림 그려야겠구나 생각 을해 돼요 이런 식 각 비 구십 도인 직각삼각형 시작 각비구십 도인 직각삼각형그려이 얘기 코사인 에이 칠분 오 코사인 빗변 밑변 이었고 그게 칠분 오우리 그림 그려 보자 빗변 밑변 이칠분 오 대략 느낌 이런 식 여러분 그려줄수있으면 돼요 비 가구도래에이 기준 각에이쪽놔볼까요그럼이거자동씨되겠죠삼각형이비했으니까코사인기준각대해마주보는게높이고이밑변코사인빗변밑변길이길이그게 칠분 오라얘기칠분오하순간우리 뭐 해야 돼 피타고라스 정리 해서구해 됩니다 비씨 루트 빗변칠제곱사십구빼기나머지변오제곱 루트 이십사이 루트

google
  칠 선택 지에서 정답 몇번  이렇게 남볼수있으면 되겠죠 담육번더짧을일어서가니까문제삼각비를쓰고그냥 그

...

그렇다면 우리 사인 코사인 에이 에이 에이 에이 뭐 코사인 코사인 높이 밑변 엑스 에이치 지금 코사인 물론 뭐 비 에이치 마이너스 빗변 곱하기 삼

**************************************************
ebs
 마지막 탄젠트 삼십 도의 값 그럼 뭐 되나 살펴보면 탄젠트 삼십 는 밑변높이잖아요그렇죠 루트 삼 에이 에이에이약분 나면 루트 삼분 일 또는 유리화해서삼분루트삼이 된다는 거 그럼오늘결론뭐여러분사인 삼십 얼마 그러면이분의일 나와야 된다는얘기코사인 삼십 얼마 이분 루트 삼 나와야되는거예요탄젠트삼십얼마삼분루트삼이렇게바로바로나올수있게끔암기되어있는게오늘결론마지막해야할일입니다

google
 마지막 탄젠트 삼십 도의 값 그럼 뭐 되냐고 살펴보면 탄젠트 삼십 는 및피자해보죠 루트 삼 에이 내리니까팔분 나면 루트 삼분  또는 유료삼분이삼일 된다는 거 오늘결론뭐여러분싸인삼십얼마그러면이분와야된다는얘기코사인 삼십 얼마 이분루트삼 나와야 되는거예요탄젠트 삼십 얼마 삼분 루트 삼 이렇게바로바로나올수있게끔암기되어있는게오늘결론마지막회그렇다면이제그림기준가를육십도로바꿔보겠습니다

predict
마지막 사인 커져서길이예들어일점이분길이 변 높이 에이 에이 길이 의비 넓이 의미 있냐면 하나 앞 배웠던 닮음비 삼각형 변 길이 뭐 둘 오면 길이 다 그러면 전체 코사인 몇 있느냐그렇진 코사인 배웠죠그렇죠각 이렇게 여기 빗변밑변의비주어져있으 돼 뭐

**************************************************
ebs
 얘기 뭘구하겠다는 얘기 사인 육십 코사인 육십 탄젠트 육십 도의 값 알아보자는 이야기 겠죠잘보세요기준각이이 쪽 바뀌면마주보변 뭐 돼 높이 되고 이변 밑변 이변 빗변 지난시 내용그렇죠 그렇다면 사인육십 도라는건 빗변 높이 빗변 높이 뭐 에이 루트 삼 에이 이렇게되겠죠에이루트삼에이에이약분이분루트삼이바사인육십도의값입니다

google
 얘기 몰고하겠단 얘기 싸인 육십 코사인 육십 탄젠트 육십  값 우리알아보자나는 이야기 

여기 꼭 탄젠트 삼십 코사인 에이 에이 에이치 뭐 보는 경우 빗변밑변의비주어져있으 양변 탄젠트 코사인 탄젠트 거 코사인 가지 오른쪽 게 밑변건뭐 사인 이렇게 빗변 빗변

**************************************************
ebs
 선생님 이런 문제안 나온다면서요그냥 외워서 쓰는거죠 근데 한번 여러분 외웠는지 다시 한번 확인 그런 문제 입니다

google
 생 이런 문제안 나온다어그 외워서 쓰는거 근데 한번 여러분 여러분왜왔는지 다시 한번 확인 그럼 문제 입니다

predict
생 사인 코사인 에이 얼마 되어있지만 의미 이렇게우리 자르지 해서 여기 각 여러분 코사인 육십 어떻게

**************************************************
ebs
 가끔 이런 문제시험 출제 되기 도합 니

google
 가끔 이런 문제점 출제 되기 도합 니

predict
가끔 우리 최종 걸렸죠 중요한건 생각 이렇게

**************************************************
ebs
 근데 여러분 들이잘틀리진 않아요 성실히 외웠다면 같이 함께 예제일번가볍게풀어볼텐데 탄젠트 사십오 얼마 였어요 표 머릿속 그려 봐 여러분 이연상해외울때탄젠트 마지막 줄이었지 사십오 는 가운데 였지 그럼 뭐 떠올라야 돼 일떠올라야 돼 루트 삼 나누고루트삼곱했던거기억해야죠이런식여러분표구개숫자다쓸수있지만머릿속이미지그려놓는연습중요합니다

google
  여러분 틀리지 않아요 성실히 왜없다면 같이 함께 삼일볼텐데 탄젠트 사십오 얼마 했어요 표 머릿속 그려 봐 여러분 연상을해배울게아편전쟁 마지막 드렸지 사십오 는 가운데 어찌 그럼 뭐 떠올라가야 돼 가야 돼 루트 삼 나누기했던걸기억해야죠이런소리런드리표구케이스닫았을수도있지만머리속이미지그려놓는연습중요합니다

predict
여러분 제 앞 답변 함께 마무리 공부 선생님 한가지 기억 여러분 여기 드리도록 문제 인터넷 숙제 여러분 형성평가 넓이 우리 기준 드릴 높이는 되

이번 선생님 오번 시작 해볼텐데요 변 정도 선생님 구해 할수있겠고 들어가서우리 지금 앞 각 아셨죠 여러분 예제 시간 높이 그림 이렇게 할거예요 방법 이렇게 이렇게 철저하게 코사인 사인 이어질거기 끝나요 에일 삼분이예만약탄젠트십오써놓으세요 여러분 이구 가눈으로 둘 여러분 문제 이렇게 이렇게 사 우리 정답 여기 않고

**************************************************
ebs
 아셨죠 제 유제사 번 문제 들어가야되 겠죠 유제사 번 문제 들어가 보겠습니다

google
 아셨죠 제  번 문제 들어가야되 겠죠  번 사번문제들어가야되겠죠자유의사번 들어가 보겠습니다

predict
아셨죠 사인 보통 탄젠트 초 반지름 길이 의비 표현 같으니까

**************************************************
ebs
 어려워 보이 굉장히 간단해요 우리 단순화 시켜서 사인 몇 도 이분 루트 삼 인지 암기되어있으면 해결 할수있어요사인 순서대로 이분 루트 일루트이 루트 삼 이었어 이분 루트 삼은 그러면 결국 사인 육십 도의 값 이이분 루트삼이었지이것만 외워져있다면 똑같고사인사 똑같으니까 삼각 엑스 더하기십 오도 육십 도 것 알 끝나겠죠 여기 엑스 구하란얘기잖아그렇죠결국삼각엑스십오항해사십오양변삼나누면각엑스크기몇가된다

google
 어려워 보이 굉장히 간다 우리 단순화 시켜서 싸인 몇 도 이분 루트 삼 인지 게되어있어 해결 할수있어쌓이는 순서대로 이분 루트 백이십삼이었어이분 루트 삼 이었어 이분 루트 삼은 그러면 결국 까인 육십  값 이이분 루트삼이어질아는것만여러분 외워져있다면 똑같고싸인싸인 똑같으니까 익스플러스이십오도육십도구것알끝나겠죠여기 엑스  오도 육십 도구 것 알 끝나겠죠 여기 엑스 십오이상사십오양변삼나누면엑셀크기몇가된다

predict
어려워 실제 우리 함께 번 삼각형 씨디 함께 오늘 우리 반지름 뭐 사이 이렇게 등변사다리꼴 꼭 당연히 둘 차이점 길이갖다가에이해보고이길이 길이그게삼분사구나이말그림이렇게표현될수

육이 반지름 길이 지금 엑스 사인코사인따져요이경우따져요 보세요 결국 결국 우리 면 삼각형 여기 이렇게 변 길이 이야기 코사인 했는데 에이 했는데 각 작은 길이 몇 틀렸다고 그렇죠 해서에이비길이구할수있잖아요 비사 써주면되 왼쪽 왼쪽 의사 해서 백오도 에이 한번 삼각형이비디 해서

**************************************************
ebs
 걸 생각 하면서 문제 읽어 볼게요 비디길이구하라했잖아요그렇죠주황색 오른쪽 있는이 삼각형 빗변 구하라이야기 그러면 일단 삼각형 어느 변 거 거 하나 알아야 된다는얘기 래야 빗변 구 하겠죠 그럼 당연히이 걸구하나아니죠 비씨 구하겠죠 왜 하얀색 넘어와야 되니까 그러면 보는거야 기준 각 육십 잖아요 그럼 지금 마주 보는게 높이 하얀색 그렇죠이건뭐밑변밑변 높이는 뭐였죠 탄젠트 입니다 그럼 탄젠트 육십 가뭐뭐입 육분 비씨 여기 우리 비씨 구 할거고 탄젠트육십는 루트 삼이 육배해주면 비씨 결국 뭐 돼 육루트삼이될거야이렇게비씨구단계들어가야죠그러면이제그다음오른쪽넘어올수있겠네다리건너봅시다비디길이구하라했습니다

google
 걸 생각 하면서 문제 읽어 볼게요 비밀뭐되잖아요그이함대 오른쪽 있는이상가평빗변구얘기그러면일단 삼각형 빗변 구얘기 그러면 일단 삼각형 어느 별일 거 거 거든하나 알아야 된다 래야 빗변 구 하겠어 그럼 당연히이 걸구하라아니죠 비씨  왜 하얀색 써넘어가야 되니까 그러면 보는거야 기준 각 육십 잖아요 그럼 지금 마주 보이는게 높이 하얀색 이상죠거밑편의점그럼이별 높이는 모였어 탄젠트 다 그럼 탄젠트 육십 가오뭡 육분 비씨 여기 우리 비씨 구 할거고  루트 삼이 육대주비씨결국뭐돼노트삼될거야이렇게 비씨 결국 뭐 돼 노트삼될거야이렇게비씨구단계들어가야죠그러면이제그다음오른쪽넘어올수있겠네다리건너목사비리바라겠습니다

predict
걸 선생님 오늘 뉴런 수학 인터넷 강의 가눈으로 부터약간이런 있습니다 인터넷 여러분 많이 보면 피타고라스 삼각형 여러분 엑스 선생님 어디 싶은데 알 시간 여러분 나와요 원칙 값 육

IndexError: list index out of range

In [ ]:
print

In [10]:
print(sentence_generation(model, t, '다시', 10))
# 다시 한번 첫 시작 지점 뭐라고 이렇게 둘 더한것이십육이 : EBS
# 다시 한번 첫 시작 기점 뭐라고 이렇게 둘 도아것이십육일이 : GOOGLE

다시 한번 문제 보도록 하겠습니다 에이 알아야해에이에이치알아야해말겠네그렇죠 당연히 아니에요 일이이 모양


In [11]:
print(sentence_generation(model, t, '주황색', 10))
# 주황색 변해요 길어지죠 빗변 길이 어떻게 되죠 길어집니다
# 주황색 변해요 빗변 길이 너어떻게 되죠 길어집니다

주황색 반지름 선생님 뭐 적용 모두 삼각비 아니라 여러분 뭐 당연히
